In [ ]:
##항공권 가져오기
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import os
import time
import re
from datetime import datetime, timedelta

chorme_options = Options()
chorme_options.add_experimental_option("detach",True)

chorme_options.add_experimental_option("excludeSwitches",["enable-logging"])

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chorme_options)

driver.implicitly_wait(5)
driver.maximize_window()
driver.get("https://www.google.com/travel/flights")

way = driver.find_element(By.CSS_SELECTOR, "#yDmH0d > c-wiz.zQTmif.SSPGKf > div > div:nth-child(2) > c-wiz > div.cKvRXe > c-wiz > div.vg4Z0e > div:nth-child(1) > div.SS6Dqf.POQx1c > div.AJxgH > div > div.Maqf5d > div.RLVa8.GeHXyb")
way.click()

oneway = driver.find_element(By.CSS_SELECTOR, "#yDmH0d > c-wiz.zQTmif.SSPGKf > div > div:nth-child(2) > c-wiz > div.cKvRXe > c-wiz > div.vg4Z0e > div:nth-child(1) > div.SS6Dqf.POQx1c > div.AJxgH > div > div.Maqf5d > div.RLVa8.GeHXyb > div > div > div > div.VfPpkd-xl07Ob-XxIAqe.VfPpkd-xl07Ob.VfPpkd-YPmvEd.s8kOBc.dmaMHc > ul > li:nth-child(3)")
oneway.click()

time.sleep(1)

origin_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[aria-label='목적지가 어디인가요? ']"))
)
#서울 / 상하이 / 도쿄 / 하노이 / 시드니 / 뉴욕 / 런던 / 베를린 / 파리 / 오클랜드
origin_input.clear()
origin_input.send_keys("서울")
origin_input.send_keys(Keys.TAB)

real_input = driver.find_element(By.CSS_SELECTOR, "#c1")
real_input.click()

time.sleep(1)
startday = driver.find_element(By.CSS_SELECTOR, "#yDmH0d > c-wiz.zQTmif.SSPGKf > div > div:nth-child(2) > c-wiz > div.cKvRXe > c-wiz > div.vg4Z0e > div:nth-child(1) > div.SS6Dqf.POQx1c > div.AJxgH > div > div.rIZzse > div.bgJkKe.K0Tsu")
startday.click()

time.sleep(1)
dayin = driver.find_element(By.CSS_SELECTOR, "#ow9 > div.ZGEB9c.yRXJAe.P0ukfb.icWGef.bgJkKe.BtDLie.iWO5td > div > div.rj7qGc.ksI2Bc.P0ukfb > div.qxcyof.RNniQb > div > div > div.SJyhnc > div > div:nth-child(2) > div:nth-child(3) > div.yg1Os.vun5vf > div.WhDFk.Io4vne.inxqCf")
dayin.click()
time.sleep(1)
daysearch = driver.find_element(By.CSS_SELECTOR, "#ow9 > div.ZGEB9c.yRXJAe.P0ukfb.icWGef.bgJkKe.BtDLie.iWO5td > div > div.akjk5c.FrVb0c > div.WXaAwc")
daysearch.click()

startcon = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[aria-label='출발지가 어디인가요?']"))
)
startcon.clear()
time.sleep(1)
startcon.send_keys("오클랜드")
time.sleep(2)
depin = driver.find_element(By.CSS_SELECTOR,'[data-code="/m/012ts"]')
depin.click()
startcon.send_keys(Keys.ENTER)

def parse_aria_label(text):
    try:
        if "최저가는" in text:
            price = text.split("최저가는")[1].split("대한민국")[0].strip()
        else:
            price = "-"
        if "항공의" in text:
            airline = text.split("항공의")[0].split()[-1] + "항공"
        else :
            airline = "-"
        if "항공편입니다" in text:
            stop_info = text.split("항공의")[1].split("항공편입니다.")[0].strip()
        else :
            stop_info = "-"
        if "출발하여" in text:
            match = re.search(r"[가-힣]+,\s*\d+월\s*\d+\s+(오전|오후)\s*\d+:\d+", text)
            if match:
                departure_time = match.group().replace(",", "")
            else:
                departure_time = "-"
        else:
            departure_time = "-"

        if "에서 출발하여" in text:
            departure_airport = text.split("에서 출발하여")[0].split("에")[-1].strip()
        else :
            departure_airport = "-"
        if "출발하여" in text:
            arrival_time = text.split("출발하여")[1].split("에")[0].strip().replace(",", "")
        else :
            arrival_time = "-"
        if "에 도착합니다" in text:
            arrival_airport = text.split("에 도착합니다")[0].split("에")[-1].strip()
        else :
            arrival_airport = "-"
        if "총 비행 시간은" in text:
            total_flight_time = text.split("총 비행 시간은")[1].split("입니다")[0].strip()
        else : total_flight_time = "-"
        if "기착 시간" in text:
            layover_section = text.split("기착 시간")[1]
            layover_airport = layover_section.split("은")[1].split("에서")[0].strip()
            layover_duration = layover_section.split("에서")[1].split("입니다")[0].strip()
        else:
            layover_airport = "직항"
            layover_duration = "-"

        return {
            "가격": price,
            "항공사": airline,
            "경유정보": stop_info,
            "출발시간": departure_time,
            "출발공항": departure_airport,
            "도착시간": arrival_time,
            "도착공항": arrival_airport,
            "총비행시간": total_flight_time,
            "경유지": layover_airport,
            "경유시간": layover_duration
        }

    except Exception as e:
        print("파싱 실패:", e)
        return None
    
parsed_list = []

elements = driver.find_elements(By.CSS_SELECTOR, 'div.JMc5Xc')

for el in elements:
    label = el.get_attribute("aria-label")

    if label:
        print("📝 수집한 label:\n", label)
        info = parse_aria_label(label)
        if info:
            parsed_list.append(info)
        else:
            print("⚠️ 파싱 실패 label:\n", label)

today = datetime.today()
total_list = []
for i in range(7):
    target_date = today + timedelta(days=i)
    date_str = target_date.strftime("%Y-%m-%d")
    parsed_list = []

    try:
        calendar_btn = driver.find_element(By.CSS_SELECTOR, "div.bgJkKe.K0Tsu")
        calendar_btn.click()
        time.sleep(1)

        day_elem = driver.find_element(By.CSS_SELECTOR, f'[data-iso="{date_str}"]')
        day_elem.click()
        time.sleep(1)

        confirm = driver.find_element(By.CSS_SELECTOR, "div.akjk5c.FrVb0c > div.WXaAwc")
        confirm.click()

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.JMc5Xc'))
        )
        time.sleep(1)

        elements = driver.find_elements(By.CSS_SELECTOR, 'div.JMc5Xc')
        for el in elements:
            label = el.get_attribute("aria-label")
            if label:
                info = parse_aria_label(label)
                if info:
                    info["출발일"] = date_str
                    parsed_list.append(info)
                    total_list.append(info)

    except Exception as e:
        print(f"처리 실패:", e)

if total_list:
    df = pd.DataFrame(total_list)
    save_path = "C:/MBC12AI/spring/flight/flight.csv"
    file_exists = os.path.exists(save_path)

    df.to_csv(
        save_path,
        index=False,
        encoding="utf-8-sig",
        mode="a",             
        header=not file_exists    
    )
    print(f"저장 완료: {save_path}")
    total_list = []






In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import time
import os
import pandas as pd

options = Options()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)
#서울 / 상하이 / 도쿄 / 하노이 / 시드니 / 뉴욕 / 런던 / 베를린 / 파리 / 오클랜드
city = "오클랜드"


driver.get("https://www.google.com/travel/search")
time.sleep(2)


search_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[aria-label='장소, 호텔 등 검색']"))
)
search_input.clear()
search_input.send_keys(city)
time.sleep(1)
travle_cl = driver.find_elements(By.CSS_SELECTOR,"li.Q1RWxd")
if travle_cl:
    travle_cl[0].click()
time.sleep(5)

parsed_list = []

today = datetime.today()
for i in range(7):
    checkin = today + timedelta(days=i)
    checkout = checkin + timedelta(days=1)
    checkin_str = checkin.strftime("%Y-%m-%d")
    checkout_str = checkout.strftime("%Y-%m-%d")

    try:
        calendar_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "[aria-label='체크인']"))
        )
        calendar_button.click()
        time.sleep(2)

        in_elem = driver.find_element(By.CSS_SELECTOR, f'[data-iso="{checkin_str}"]')
        in_elem.click()
        time.sleep(1)

        out_elem = driver.find_element(By.CSS_SELECTOR, f'[data-iso="{checkout_str}"]')
        out_elem.click()
        time.sleep(1)

        confirm_btn = driver.find_element(By.CSS_SELECTOR, 'button[jsname="iib5kc"]')
        confirm_btn.click()
        time.sleep(5)

        hotels = driver.find_elements(By.CSS_SELECTOR, 'li.fczSu')

        for hotel in hotels:
            try:
                name = hotel.find_element(By.CSS_SELECTOR, 'div.AdWm1c.AFZtd.ogfYpf.AF5RIe').text
                img = hotel.find_element(By.TAG_NAME, 'img')
                img_url = img.get_attribute("src") or img.get_attribute("data-src")
                price = hotel.find_element(By.CSS_SELECTOR, 'div.UydQr.AdWm1c').text 
                link = hotel.find_element(By.TAG_NAME, 'a').get_attribute('href')

                parsed_list.append({
                    "도시": city,
                    "체크인": checkin_str,
                    "체크아웃": checkout_str,
                    "호텔명": name,
                    "이미지": img_url,
                    "가격": price,
                    "링크": link
                })
            except Exception as e:
                print(f"추출 오류: {e}")
    except Exception as e:
        print(f"처리 실패:", e)

if parsed_list:
    df = pd.DataFrame(parsed_list)
    os.makedirs("C:/MBC12AI/spring/hotels", exist_ok=True)

    file_path = "C:/MBC12AI/spring/hotels/hotel_list.csv"
    file_exists = os.path.exists(file_path)

    df.to_csv(
        file_path,
        index=False,
        encoding="utf-8-sig",
        mode="a",
        header=not file_exists
    )
    print(f"저장 완료: {file_path}")
else:
    print("저장할 숙소 정보가 없습니다.")

✅ 숙박 정보 누적 저장 완료: C:/MBC12AI/spring/hotels/hotel_list.csv
